In [1]:
!pip install -q deepface ipywidgets pillow opencv-python


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')


In [4]:
import io
import random
from base64 import b64decode
from collections import Counter
from PIL import Image, ImageOps, ImageEnhance, ImageFilter

from deepface import DeepFace
import ipywidgets as widgets
from IPython.display import display, clear_output

# Colab webcam support check
try:
    from google.colab.output import eval_js
    _HAS_COLAB = True
except Exception:
    _HAS_COLAB = False

# ---------- Content ----------
jokes = [
    "Why did the math book look sad? Because it had too many problems!",
    "Why don’t skeletons fight each other? They don’t have the guts.",
    "Why did the computer catch a cold? It had a bad byte!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "I told my computer I needed a break, and it said 'No problem — I'll go to sleep.'",
    "Why was the smartphone wearing glasses? It lost its contacts.",
    "Why did the bicycle fall over? Because it was two tired!",
    "Why do bees have sticky hair? Because they use honeycombs!",
    "What did one ocean say to the other? Nothing — they just waved.",
    "I asked the librarian if the library had books on paranoia. She whispered, 'They're right behind you.'",
    "Why don't programmers like nature? Too many bugs.",
    "Why did the cookie go to the doctor? Because it felt crummy.",
    "Why did the tomato blush? Because it saw the salad dressing!",
    "What do you call fake spaghetti? An impasta!",
    "Why did the golfer bring two pairs of pants? In case he got a hole in one.",
    "Parallel lines have so much in common. It’s a shame they’ll never meet."
]

quotes = [
    "Keep your face always toward the sunshine—and shadows will fall behind you.",
    "The only way to do great work is to love what you do.",
    "You are capable of amazing things!",
    "Difficult roads often lead to beautiful destinations.",
    "Small steps every day lead to big changes.",
    "You are stronger than you think — one breath at a time."
]

# ---------- UI Style Helpers ----------
def make_card(content, bg='#f9faff', color='#1e6fb3', radius='16px', shadow='0 4px 20px rgba(30,111,179,0.15)', padding='16px 20px', margin='12px 0'):
    return widgets.HTML(
        f"""<div style="
            background: {bg};
            color: {color};
            border-radius: {radius};
            box-shadow: {shadow};
            padding: {padding};
            margin: {margin};
            font-size: 17px;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        ">
            {content}
        </div>"""
    )

def pill_button(description, style, icon=None):
    return widgets.Button(
        description=description,
        button_style=style,
        layout=widgets.Layout(width='180px', height='46px', border_radius='30px'),
        icon=icon if icon else ""
    )

# ---------- Helpers ----------
def pil_to_widget_image(pil_img, max_size=(220,220), fmt='JPEG'):
    img = pil_img.copy()
    img.thumbnail(max_size)
    buf = io.BytesIO()
    img.save(buf, format=fmt)
    return widgets.Image(value=buf.getvalue(), format=fmt.lower(), width=img.width, height=img.height)

def _enhance_image_bytes(image_bytes, save_path="temp_enhanced.jpg"):
    try:
        img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    except Exception:
        with open(save_path, "wb") as f:
            f.write(image_bytes)
        return save_path

    img = ImageOps.autocontrast(img, cutoff=1)
    img = ImageEnhance.Contrast(img).enhance(1.20)
    img = img.filter(ImageFilter.SHARPEN)
    img.save(save_path, format="JPEG", quality=90)
    return save_path

def _pick_best_from_scores(emotion_scores):
    if not emotion_scores:
        return None
    return max(emotion_scores.items(), key=lambda kv: kv[1])[0]

def detect_emotion_improved(image_bytes, backends=None, temp_path="temp_enhanced.jpg"):
    if backends is None:
        backends = ["retinaface", "mtcnn", "ssd", "opencv"]

    img_path = _enhance_image_bytes(image_bytes, save_path=temp_path)
    votes = []
    confidences = []
    debug = {"backend_results": {}}

    for b in backends:
        try:
            res = DeepFace.analyze(img_path=img_path, actions=["emotion"], detector_backend=b, enforce_detection=False)
        except Exception as e:
            debug["backend_results"][b] = {"error": str(e)}
            continue

        entry = None
        if isinstance(res, list) and len(res) > 0 and isinstance(res[0], dict):
            entry = res[0]
        elif isinstance(res, dict):
            entry = res

        if not entry:
            debug["backend_results"][b] = {"error": "unexpected response", "raw": str(res)}
            continue

        dominant = entry.get("dominant_emotion") or entry.get("dominantEmotion")
        emotion_scores = entry.get("emotion") or {}

        if not dominant and emotion_scores:
            dominant = _pick_best_from_scores(emotion_scores)

        top_score = None
        if emotion_scores and dominant in emotion_scores:
            try:
                top_score = float(emotion_scores[dominant])
            except Exception:
                top_score = None

        if dominant:
            d = str(dominant).lower()
            votes.append(d)
            confidences.append((d, top_score))
            debug["backend_results"][b] = {"dominant": d, "score": top_score}
        else:
            debug["backend_results"][b] = {"dominant": None, "score": None}

    if not votes:
        return "Neutral", {"reason": "no_backend_success", "debug": debug}

    counts = Counter(votes)
    scores_by_emotion = {}
    counts_by_emotion = Counter()
    for emo, score in confidences:
        counts_by_emotion[emo] += 1
        if score is not None:
            scores_by_emotion.setdefault(emo, []).append(float(score))

    if scores_by_emotion:
        mean_conf = {emo: (sum(vals) / len(vals)) for emo, vals in scores_by_emotion.items()}
        best = max(mean_conf.items(), key=lambda kv: (kv[1], counts_by_emotion[kv[0]]))[0]
        final = best
    else:
        final = counts.most_common(1)[0][0]

    return str(final).capitalize(), {"debug": debug, "votes": list(votes), "counts": dict(counts)}

detect_emotion = detect_emotion_improved

In [8]:

# ---------- Webcam Capture -----------
def take_photo_with_webcam(quality=0.8):
    if not _HAS_COLAB:
        raise RuntimeError("Webcam capture is only available in Google Colab.")

    js = f"""
    async function capture(quality) {{
      const div = document.createElement('div');
      div.style.position = 'fixed';
      div.style.left = '10px';
      div.style.top = '10px';
      div.style.zIndex = 99999;
      div.style.background = 'rgba(255,255,255,0.97)';
      div.style.border = '1px solid #ccc';
      div.style.padding = '14px 18px';
      div.style.borderRadius = '14px';
      div.style.boxShadow = '0 6px 18px rgba(0,0,0,0.12)';
      div.style.display = 'flex';
      div.style.flexDirection = 'column';
      div.style.alignItems = 'center';

      const video = document.createElement('video');
      video.style.width = '360px';
      video.style.borderRadius = '12px';
      video.style.boxShadow = '0 4px 8px rgba(30,111,179,0.2)';
      video.style.display = 'none';

      const controls = document.createElement('div');
      controls.style.display = 'flex';
      controls.style.justifyContent = 'center';
      controls.style.gap = '12px';
      controls.style.marginTop = '14px';

      const startBtn = document.createElement('button');
      const captureBtn = document.createElement('button');
      const cancelBtn = document.createElement('button');
      startBtn.textContent = 'Start Camera';
      captureBtn.textContent = 'Capture';
      cancelBtn.textContent = 'Cancel';

      captureBtn.style.display = 'none';
      captureBtn.style.padding = '8px 18px';
      captureBtn.style.borderRadius = '8px';
      captureBtn.style.border = 'none';
      captureBtn.style.backgroundColor = '#1e6fb3';
      captureBtn.style.color = '#fff';
      captureBtn.style.fontWeight = '600';

      startBtn.style.padding = '8px 18px';
      startBtn.style.borderRadius = '8px';
      startBtn.style.border = 'none';
      startBtn.style.backgroundColor = '#1e6fb3';
      startBtn.style.color = '#fff';
      startBtn.style.fontWeight = '600';

      cancelBtn.style.padding = '8px 18px';
      cancelBtn.style.borderRadius = '8px';
      cancelBtn.style.border = '1px solid #ccc';
      cancelBtn.style.backgroundColor = '#fff';
      cancelBtn.style.color = '#666';

      div.appendChild(video);
      controls.appendChild(startBtn);
      controls.appendChild(captureBtn);
      controls.appendChild(cancelBtn);
      div.appendChild(controls);
      document.body.appendChild(div);

      let stream = null;

      startBtn.onclick = async () => {{
        try {{
          stream = await navigator.mediaDevices.getUserMedia({{video: true, audio: false}});
          video.srcObject = stream;
          await video.play();
          video.style.display = 'block';
          captureBtn.style.display = 'inline-block';
          startBtn.textContent = 'Restart Camera';
        }} catch (err) {{
          div.remove();
          throw(err);
        }}
      }};

      const p = new Promise((resolve, reject) => {{
        captureBtn.onclick = () => {{
          try {{
            if (!stream) {{ reject('camera not started'); return; }}
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(t => t.stop());
            const data = canvas.toDataURL('image/jpeg', quality);
            div.remove();
            resolve(data);
          }} catch (e) {{
            reject(e);
          }}
        }};
        cancelBtn.onclick = () => {{
          if (stream) {{ stream.getTracks().forEach(t => t.stop()); }}
          div.remove();
          resolve(null);
        }};
      }});
      return p;
    }}
    capture({quality});
    """

    data = eval_js(js)
    if not data:
        raise RuntimeError("No image captured (user cancelled or camera not available).")

    header, b64 = data.split(',', 1)
    return b64decode(b64)

# ---------- Tic Tac Toe Game ----------
def tic_tac_toe_game():
    board = ['']*9
    current = ['X']
    game_over = [False]
    result_out = widgets.Output(layout=widgets.Layout(min_height='40px'))

    def check_winner():
        wins = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
        for a,b,c in wins:
            if board[a] and board[a]==board[b]==board[c]:
                return board[a]
        if all(board):
            return 'Draw'
        return None

    def make_onclick(i):
        def _onclick(btn):
            if not board[i] and not game_over[0]:
                board[i] = current[0]
                buttons[i].description = current[0]
                winner = check_winner()
                if winner:
                    game_over[0] = True
                    with result_out:
                        clear_output()
                        if winner == 'Draw':
                            print("It's a draw!")
                        else:
                            print(f"Player {winner} wins! 🎉")
                else:
                    current[0] = 'O' if current[0] == 'X' else 'X'
        return _onclick

    btn_layout = widgets.Layout(width='80px', height='80px')
    buttons = [widgets.Button(description='', layout=btn_layout) for _ in range(9)]
    for i, b in enumerate(buttons):
        b.on_click(make_onclick(i))

    grid = widgets.GridBox(buttons, layout=widgets.Layout(grid_template_columns='repeat(3, 80px)', grid_gap='8px'))
    reset_btn = widgets.Button(description='Reset', button_style='success', layout=widgets.Layout(width='100px', height='36px'))
    def reset_game(b):
        for i in range(9):
            board[i] = ''
            buttons[i].description = ''
        game_over[0] = False
        current[0] = 'X'
        with result_out:
            clear_output()
    reset_btn.on_click(reset_game)

    header = widgets.HTML("<div style='font-weight:700; font-size:18px; color:#1e6fb3;'>Tic‑Tac‑Toe</div>")
    sub = widgets.HTML("<div style='color:#555; font-size:14px; margin-bottom:8px;'>Play with your friend — take turns tapping the squares.</div>")
    card = widgets.VBox([header, sub, grid, widgets.HBox([reset_btn, result_out])],
                        layout=widgets.Layout(border='1px solid #d1e0ff', padding='16px', width='360px', align_items='center', box_shadow='0 6px 15px rgba(30,111,179,0.1)'))
    return card

# ---------- UI Actions ----------
def disable_controls():
    joke_btn.disabled = True
    quote_btn.disabled = True
    game_btn.disabled = True
    photo_btn.disabled = True
    uploader.disabled = True

def enable_controls():
    joke_btn.disabled = False
    quote_btn.disabled = False
    game_btn.disabled = False
    photo_btn.disabled = False
    uploader.disabled = False

def safe_unobserve():
    try:
        uploader.unobserve(on_image_upload, names='value')
    except Exception:
        pass

def safe_observe():
    try:
        uploader.observe(on_image_upload, names='value')
    except Exception:
        pass

# Display Debug Info
def show_debug_info(info):
    out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='12px', max_height='300px', overflow='auto'))
    with out:
        import json
        print("Debug info (raw):")
        print(json.dumps(info, indent=2))
    clear_output(wait=True)
    display(feature_card)
    display(out)
    display(control_row)

# Cheer Up UI based on mood
def cheer_up_ui(emotion, debug_info=None):
    clear_output(wait=True)
    display(feature_card)

    color_map = {
        'happy': '#2d7a2d',
        'neutral': '#1f6fb3',
        'sad': "#da42ac",
        'angry': '#b22828',
        'fear': "#a85ee0",
        'disgust': '#b22828',
        'surprise': '#f39c12',
        'unknown': '#666666'
    }
    base_emotion = emotion.lower()
    color = color_map.get(base_emotion, '#1f6fb3')

    mood = widgets.HTML(f"<div style='font-size:20px; font-weight:700; color:{color}; margin:10px 0;'>Detected Mood: {emotion}</div>")
    display(mood)

    if debug_info:
        btn = widgets.Button(description="Show Detection Debug Info", button_style='info', layout=widgets.Layout(width='220px', margin='12px 0'))
        btn.on_click(lambda b: show_debug_info(debug_info))
        display(btn)

    negatives = ['sad', 'disgust', 'angry', 'fear', 'unknown']
    if base_emotion in negatives:
        info = widgets.HTML("<div style='font-size:15px; color:#fafafa; background:#b22828; padding:10px; border-radius:10px; margin:12px 0;'>I detected a low mood — choose a joke, a quote, or play a quick game to cheer up!</div>")
        display(info)
        display(control_row)
    else:
        happy = widgets.HTML("<div style='font-size:16px; color:#2d7a2d;'>You look OK — keep smiling! 😊</div>")
        display(happy)
        display(control_row)

# Show joke, quote, game UI
def show_joke(_=None):
    clear_output(wait=True)
    display(feature_card)
    title = widgets.HTML("<h3 style='margin-bottom:8px;'>Here's a joke to cheer you up:</h3>")
    joke_html = f"""
    <div style="background:linear-gradient(90deg,#fff8e6,#fffaf0);
                padding:16px; border-radius:14px; border:1px solid #ffe0a8;
                color:#6a3b00; font-size:18px; font-family:'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
      <b style='display:block; margin-bottom:10px; color:#c65a00;'>😁 Light moment</b>
      {random.choice(jokes)}
    </div>
    """
    display(widgets.VBox([title, widgets.HTML(joke_html)], layout=widgets.Layout(margin='0 0 16px 0')))
    display(control_row)

def show_quote(_=None):
    clear_output(wait=True)
    display(feature_card)
    title = widgets.HTML("<h3 style='margin-bottom:8px;'>A motivational quote:</h3>")
    quote_html = f"""
    <div style="background:#eafaf1; padding:16px; border-radius:14px; border:1px solid #cfead6;
                color:#0b6b3a; font-size:18px; font-family:'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
      <b style='display:block; margin-bottom:10px;'>🌱 Thought for today</b>
      {random.choice(quotes)}
    </div>
    """
    display(widgets.VBox([title, widgets.HTML(quote_html)], layout=widgets.Layout(margin='0 0 16px 0')))
    display(control_row)

def show_game(_=None):
    clear_output(wait=True)
    display(feature_card)
    display(tic_tac_toe_game())
    display(control_row)

# Upload image handler
def on_image_upload(change):
    new_files = change.get('new') or ()
    if not new_files:
        return

    safe_unobserve()
    disable_controls()
    try:
        clear_output(wait=True)
        display(feature_card)

        if isinstance(new_files, dict):
            files_iter = new_files.items()
        elif isinstance(new_files, (tuple, list)):
            files_iter = [(None, f) for f in new_files]
        else:
            files_iter = []

        for fname, fileinfo in files_iter:
            pil_img = Image.open(io.BytesIO(fileinfo['content'])).convert('RGB')
            display(make_card("<b>Uploaded image preview:</b>", bg='#e0f3ff', color='#234e7d'))
            thumb_widget = pil_to_widget_image(pil_img, max_size=(180,180))
            display(thumb_widget)
            display(make_card("<i>Analyzing mood... (may download models on first run)</i>", bg='#f0faff',color='#4484d6'))

            emotion, info = detect_emotion(fileinfo['content'])
            debug_info = info.get("debug") if isinstance(info, dict) else info
            cheer_up_ui(emotion, debug_info)
            break

    except Exception as e:
        clear_output(wait=True)
        display(feature_card)
        display(make_card(f"<b style='color:#b22828;'>Error processing image:</b> {str(e)}", bg='#ffe6e6', color='#b22828' ))
        display(control_row)
    finally:
        enable_controls()
        safe_observe()

# Webcam capture handler
def on_take_photo_clicked(b):
    safe_unobserve()
    disable_controls()
    try:
        clear_output(wait=True)
        display(feature_card)
        display(make_card("Opening webcam — allow camera permission. Click 'Start Camera', then 'Capture'.", bg='#f0faff', color='#1e6fb3'))
        display(control_row)
        if not _HAS_COLAB:
            display(make_card("Webcam capture works only in Google Colab. Please use Upload instead.", bg='#ffe6e6', color='#b22828'))
            return
        img_bytes = take_photo_with_webcam(quality=0.8)
        pil_img = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        display(make_card("<b>Captured image (preview):</b>", bg='#e0f3ff', color='#234e7d'))
        thumb_widget = pil_to_widget_image(pil_img, max_size=(180,180))
        display(thumb_widget)
        display(make_card("Analyzing mood...", bg='#f0faff', color='#4484d6'))
        emotion, info = detect_emotion(img_bytes)
        debug_info = info.get("debug") if isinstance(info, dict) else info
        cheer_up_ui(emotion, debug_info)
    except Exception as e:
        clear_output(wait=True)
        display(feature_card)
        display(make_card(f"<b style='color:#b22828;'>Webcam error:</b> {str(e)}", bg='#ffe6e6', color='#b22828'))
        display(control_row)
    finally:
        enable_controls()
        safe_observe()

In [ ]:
 #---------- Initialize Controls ----------#
joke_btn = pill_button("Tell me a joke", 'info', 'smile-o')
quote_btn = pill_button("Show a quote", 'success', 'leaf')
game_btn = pill_button("Play Tic‑Tac‑Toe", 'warning', 'gamepad')
photo_btn = pill_button("Take Photo (webcam)", 'primary', 'camera')

joke_btn.on_click(show_joke)
quote_btn.on_click(show_quote)
game_btn.on_click(show_game)
photo_btn.on_click(on_take_photo_clicked)

uploader = widgets.FileUpload(accept='image/*', multiple=False)
safe_observe()
uploader.observe(on_image_upload, names='value')

uploader_box = widgets.VBox([
    widgets.HTML("<div style='font-weight:600; color:#34568a; font-size:14px; margin-bottom:6px;'>Upload Image</div>"),
    uploader
], layout=widgets.Layout(align_items='center', width='220px'))

control_row = widgets.HBox([joke_btn, quote_btn, game_btn, photo_btn, uploader_box],
                          layout=widgets.Layout(gap='16px', align_items='center', margin='12px 0'))

feature_card = widgets.VBox([
    widgets.HTML(
        "<div style='background:linear-gradient(90deg,#d0e6ff,#f1f8ff); padding:16px; "
        "border-radius:16px; width:900px; font-family:Segoe UI, Tahoma, Geneva, Verdana, sans-serif;'>"
        "<div style='display:flex; align-items:center; gap:14px;'>"
        "<div style='font-size:26px; color:#1e6fb3; font-weight:700;'>😊 Mood Booster</div>"
        "<div style='color:#555; font-size:15px;'>Upload or take a photo — I'll detect your mood and cheer you up!</div>"
        "</div></div>")
], layout=widgets.Layout(margin='10px 0 16px 0'))

# ---------- Initial Render ----------
clear_output(wait=True)
display(feature_card)
display(make_card("<b>Step 1:</b> Upload a photo or take a live photo using your webcam (Colab only). The feature will detect mood and offer a friendly response.", bg='#e7f1ff', color='#1e6fb3'))
display(control_row)

print("Note: First run may download model weights for DeepFace detector backend")
print("Note: Webcam capture works in Google Colab - click 'Take Photo' and follow prompts.")


In [10]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
